In [ ]:
pip install gradio

In [2]:
import gradio as gr
import torch
import torchtext

In [3]:
cd /content/drive/MyDrive/ECE1786/A2/

/content/drive/MyDrive/ECE1786/A2


In [4]:
glove = torchtext.vocab.GloVe(name="6B",dim=100)

In [5]:
class Baseline_Model(torch.nn.Module):
  def __init__(self, vocab):
    super().__init__()

    self.embedding = torch.nn.Embedding.from_pretrained(vocab.vectors)
    self.fc = torch.nn.Linear(100, 1)

  def forward(self, x):
    e = self.embedding(x)
    out = torch.mean(e, 0).unsqueeze(0)
    out = self.fc(out).squeeze()
    return out

class CNN_Model(torch.nn.Module):
  def __init__(self, vocab, kernel_1, kernel_2, out_channel_1, out_channel_2, freeze):
    super().__init__()
    self.kernel_1 = kernel_1
    self.kernel_2 = kernel_2
    self.out_channel_1 = out_channel_1
    self.out_channel_2 = out_channel_2
    self.embedding = torch.nn.Embedding.from_pretrained(vocab.vectors, freeze=freeze)

    self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=out_channel_1, kernel_size=(kernel_1, 100),bias=False)
    self.conv2 = torch.nn.Conv2d(in_channels=1, out_channels=out_channel_2, kernel_size=(kernel_2, 100),bias=False)
    #self.pool = torch.nn.MaxPool2d(2,2)
    self.fc = torch.nn.Linear(out_channel_1+out_channel_2,1)
    self.do = torch.nn.Dropout(0.4)
    self.bn1 = torch.nn.BatchNorm2d(out_channel_1)
    self.bn2 = torch.nn.BatchNorm2d(out_channel_2)

  def forward(self, x):
    e = self.embedding(x)
    pool1 = torch.nn.MaxPool1d(e.shape[0]-self.kernel_1+1)
    pool2 = torch.nn.MaxPool1d(e.shape[0]-self.kernel_2+1)
    e = torch.transpose(e,0,1).unsqueeze(1)
    out1 = pool1(torch.nn.functional.relu(self.conv1(e)).squeeze())
    out2 = pool2(torch.nn.functional.relu(self.conv2(e)).squeeze())
    out1 = self.do(out1)
    out2 = self.do(out2)
    out = torch.cat((out1, out2), 1).squeeze()
    out = out.view(-1, self.out_channel_1+self.out_channel_2)
    out = self.fc(out)
    
    return out.squeeze()

In [ ]:
model_base = Baseline_Model(glove)
model_base.load_state_dict(torch.load('model_baseline.pt'))
model_base.eval()
model_cnn = CNN_Model(glove, 2, 4, 50, 50, False)
model_cnn.load_state_dict(torch.load('model_cnn.pt', map_location='cpu'))
model_cnn.eval()

In [20]:
def predict(sentence):
    tokens = sentence.split()
    token_ints = [glove.stoi.get(tok, len(glove.stoi)-1) for tok in tokens]

    ### add paddings if input words are less than 5
    while len(token_ints)<5:
      token_ints.append(0)
    token_tensor = torch.LongTensor(token_ints).view(-1, 1)
    prediction_base = model_base(token_tensor)
    prediction_cnn = model_cnn(token_tensor)
    if prediction_base >= 0:
      output_base = 'Subjective'
    else:
      output_base = 'Objective'
    if prediction_cnn >= 0:
      output_cnn = 'Subjective'
    else:
      output_cnn = 'Objective'
    return output_base, output_cnn

In [18]:
demo = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=1, placeholder="Type Here..."),
    outputs=["label", "label"],
    title='Sentence Classifier',
)
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://10472.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f8451cf7f50>,
 'http://127.0.0.1:7860/',
 'https://10472.gradio.app')